In [23]:
_lambda = 399e-9
k = 2*np.pi/_lambda

TD=0.5e-3
Tr = (hbar*k)**2/m/kB

In [25]:
vrms = np.sqrt(TD/Tr)*np.sqrt(hbar*k/m)
vrms*.1e-3*1e6/20

10.194245331440493

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.constants
from qutip import *
import random
from numpy.random import choice
import matplotlib.cm as cm
from sympy import *
from IPython.display import Image
from sympy.physics.wigner import wigner_6j
from sympy.physics.quantum.cg import CG
import tqdm
from numpy import random
from scipy.optimize import curve_fit

""" 
constants
"""
pi = np.pi
c = sp.constants.c
epsilon0 = sp.constants.epsilon_0
hbar = sp.constants.hbar
h = sp.constants.h
u = sp.constants.physical_constants['atomic mass constant'][0]
kB = sp.constants.physical_constants['Boltzmann constant'][0]
mB = sp.constants.physical_constants['Bohr magneton'][0]

# α/h [Hz/(V/m)2]=2.48832×10−8 α [a.u.] from https://arxiv.org/pdf/1507.06570.pdf
# α [a.u.] = 1.648 777 274 36 x 10-41 α [C2 m2 J-1] #https://physics.nist.gov/cgi-bin/cuu/Value?auepol
a_au = 1.64877727436*1e-41

In [5]:
#monte-carlo for temp measurement
from numpy import random
from scipy.stats import norm
m = 171*u #mass
_lambda = 759e-9 #532 wavelength
w0 = 0.8e-6
zR = pi*w0**2/ _lambda #Rayleigh range
U = h*7.6e6

############
omg_r = 2*np.pi*50e3
omg_ax = omg_r*w0/zR/np.sqrt(2)

In [1]:


g = scipy.constants.g

print(omg_r/2/pi)

def KE(vx, vy, vz):
    return(m*(vx**2+vy**2+vz**2)/2)

def PE(x, y, z):
    return -U*(1 - 2*(x**2+y**2)/w0**2 - (z/zR)**2)

def scaled_prob(prob, a):
    return prob*a

# experimental data
t_array = s[0]*1e-3
exp_data = s[1]

print(t_array)

T_array = np.linspace(1e-6, 10e-6, 200)
chiT_array = []
recapture_prob_array_T = []
Ntraj = 1000
for idx in tqdm.trange(len(T_array)):
    T = T_array[idx]
    recapture_prob_array = []
    chiT = 0
    v_std = np.sqrt(kB*T/m)
    v_array = np.linspace(-5*v_std, 5*v_std, 100000)
    rpos_std = np.sqrt(kB*T/m/omg_r**2)
    rpos_array = np.linspace(-5*rpos_std, 5*rpos_std, 100000)
    axpos_std = np.sqrt(kB*T/m/omg_ax**2)
    axpos_array = np.linspace(-5*axpos_std, 5*axpos_std, 100000)
    for i in range(len(t_array)):
        dt = t_array[i]
        n=0
        recapture = 0
        
        xs = norm.rvs(size=Ntraj, scale=rpos_std)
        ys = norm.rvs(size=Ntraj, scale=rpos_std)
        zs = norm.rvs(size=Ntraj, scale=axpos_std)
        vxs = norm.rvs(size=Ntraj, scale=v_std)
        vys = norm.rvs(size=Ntraj, scale=v_std)
        vzs = norm.rvs(size=Ntraj, scale=v_std)
            
        while n<Ntraj:
            
            xi = xs[n]
            yi = ys[n]
            zi = zs[n]
            vx = vxs[n]
            vy = vys[n]
            vz = vzs[n]
                
            xf = xi + vx*dt
            yf = yi + vy*dt
            zf = zi + vz*dt - g*dt**2

            Etot = KE(vx, vy, vz) + PE(xf, yf, zf)

            if Etot < 0:
                recapture = recapture + 1
            else: 
                recapture = recapture

            n = n+1

        recapture_prob = recapture/Ntraj
        recapture_prob_array.append(recapture_prob)
        
    #guess = [exp_data[0]]
    #popt, pcov = curve_fit(scaled_prob, t_array, recapture_prob_array)
    recapture_prob_array_new = np.multiply(recapture_prob_array, exp_data[0])
        
    for i in range(len(t_array)):
        recapture_prob = recapture_prob_array_new[i]
        chi = (exp_data[i] - recapture_prob)**2 / abs(exp_data[i])
        chiT = chiT + chi
        
    chiT_array.append(chiT)
    recapture_prob_array_T.append(recapture_prob_array_new)
    
    bestT_idx = np.argmin(chiT_array)

fig, ax = plt.subplots(figsize=[5,4])
plt.plot(T_array, chiT_array, 'k-', alpha=0.7)
plt.xlabel('temp')
plt.ylabel('chi2')
plt.show()
print('smallest chi2: ', chiT_array[bestT_idx])

bestT_idx = np.argmin(chiT_array)

fig, ax = plt.subplots(figsize=[5,4])
plt.plot(t_array, exp_data, 'ko', alpha=0.7, label='data')
plt.plot(t_array, recapture_prob_array_T[bestT_idx], 'k-', alpha=0.7, label='Monte-Carlo')
plt.xlabel('time')
plt.ylabel('recapture prob')
plt.title('temperature: %.4f uK' %(T_array[bestT_idx]*1e6))
plt.show()

NameError: name 'au' is not defined